- 하이퍼파라미터: 사용자가 직접 값을 설정하는 매개변수
    - 예시: LearningRate, HiddenLayer의 크기 등
- 하이퍼파리미터 튜닝의 방법
    - 격자 검색: 각 하이퍼파라미터의 잠재적인 값의 범위를 정한 다음, 최상의 조합을 찾을 때까지 모든 값을 대입
    - scikit-optimize를 사용해 베이즈 최적화. 지정된 분포에서 고정된 갯수만큼 표본 추출 후 적용함.

In [1]:
# 사이킷런에서 와인 데이터를 가져온다.
from sklearn import datasets

wine_dataset = datasets.load_wine()
X = wine_dataset.data
Y = wine_dataset.target

In [2]:
# XGBoost와 계층화된 K-Fold를 들여온다.
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

# scikit-optimize에서 BayesSearchCV를 가져와 테스트하기 위한 매개변수 설정의 갯수를 지정
from skopt import BayesSearchCV
n_iterations = 50

# estimator를 지정. 이 경우 XGBoost를 선택하고 다계급 분류를 수행하도록 설정
estimator = xgb.XGBClassifier(
    n_jobs=-1,
    objective="multi:softmax",
    eval_metric="merror",
    verbosity=0,
    num_class=len(set(Y)),
)

D:\Programs\Anaconda\envs\AI-Study-Archive\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:

# 매개변수 검색 범위를 지정
search_space = {
    "learning_rate": (0.01, 1.0, "log-uniform"),
    "min_child_weight": (0, 10),
    "max_depth": (1, 50),
    "max_delta_step": (0, 10),
    "subsample": (0.01, 1.0, "uniform"),
    "colsample_bytree": (0.01, 1.0, "log-uniform"),
    "colsample_bylevel": (0.01, 1.0, "log-uniform"),
    "reg_lambda": (1e-9, 1000, "log-uniform"),
    "reg_alpha": (1e-9, 1.0, "log-uniform"),
    "gamma": (1e-9, 0.5, "log-uniform"),
    "min_child_weight": (0, 5),
    "n_estimators": (5, 5000),
    "scale_pos_weight": (1e-6, 500, "log-uniform"),
}
# 수행할 교차검증 유형을 지정한다.
cv = StratifiedKFold(n_splits=3, shuffle=True)


# 정의한 설정을 사용할 BayesSearchCV를 정의한다.
bayes_cv_tuner = BayesSearchCV(
    estimator=estimator,
    search_spaces=search_space,
    scoring="accuracy",
    cv=cv,
    n_jobs=-1,
    n_iter=n_iterations,
    verbose=0,
    refit=True,
)

In [4]:
# 매개변수 검색의 진행 상황을 출력할 콜백함수를 정의한다.
import numpy as np
import pandas as pd

# 지금까지 찾아온 최상의 매개변수와 검색에서 얻은 정확도를 보여준다.
def print_status(optimal_result):
    models_tested = pd.DataFrame(bayes_cv_tuner.cv_results_)
    best_parameters_so_far = pd.Series(bayes_cv_tuner.best_params_)
    print(
        "Model #{}\nBest accuracy so far: {}\nBest parameters so far: {}\n".format(
            len(models_tested),
            np.round(bayes_cv_tuner.best_score_, 3),
            bayes_cv_tuner.best_params_,
        )
    )

    clf_type = bayes_cv_tuner.estimator.__class__.__name__
    models_tested.to_csv(clf_type + "_cv_results_summary.csv")

# 매개변수 검색을 수행한다.
result = bayes_cv_tuner.fit(X, Y, callback=print_status)

ValueError: Not all points are within the bounds of the space.